In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import json
cwd = os.getcwd()

In [3]:
bestiaryFiles = os.listdir("./bestiary")
spellFiles = os.listdir("./spells")

In [4]:
def has_dice(tag):
    return tag.has_attr("data-packed-dice")

In [120]:
conditions = ['blinded','charmed','deafened','frightened','grappled','incapacitated','invisible']
conditions += 'paralyzed','petrified','poisoned','prone','restrained','stunned','exhaustion'

conditionsRegex = ""
for i, condition in enumerate(conditions):
    condition = "[" + condition[0].upper() + condition[0].lower() + "]" + condition[1:]
    conditionsRegex += condition
    if(i<len(conditions)-1):
        conditionsRegex += "|"
conditionsRegex = re.compile(conditionsRegex)

casters = 0

nums_dict = {
    'one': '1',
    'two': '2',
    'three': '3',
    'four': '4',
    'five': '5',
    'six': '6',
    'seven': '7',
    'eight': '8',
    'nine': '9',
    'zero': '0'
}

for creatureFile in bestiaryFiles:
    actions = []
    bonuses = []
    reactions = []
    features = []

    legendaryAct = 0
    legendaryRes = 0
    mythic = False
    regional = False
    lair = False
    bonus = False
    reaction = False
    action = False
    multiAttack = False
    spellCaster = False
    with open(cwd + "\\bestiary\\" + creatureFile, "r") as f:
        soup = BeautifulSoup(f,'html.parser')
        name = soup.find_all("h1", {"class": "stats-name"})[0].string
        placesOfInterest = soup.find_all("td", {"class": "mon__sect-row-inner"})
        headersOfInterest = soup.find_all("h3", {"class":"mon__sect-header-inner"})
        for header in headersOfInterest:
            if(header.string == "Legendary Actions"):
                instances = soup(text=re.compile("take \d legendary actions"))
                match = re.search(r'take \d legendary actions', instances[0])
                legendaryAct = match.group()[5]
            elif(header.string == "Mythic Actions"):
                mythic = True
            elif(header.string == "Regional Effects"):
                regional = True
            elif(header.string == "Lair Actions"):
                lair = True
            elif(header.string == "Bonus Actions"):
                bonus = True
            elif(header.string == "Reactions"):
                reaction = True
            else:
                action = True
        currentIndexOfInterest = 0
        if(len(placesOfInterest)-len(headersOfInterest)>0):
            featureList = placesOfInterest[currentIndexOfInterest]
            featureList = featureList.find_all("span",{'class': ["rd__h","rd__h--3"]})
            for featureOriginal in featureList:
                featureHTML = featureOriginal.find("span",{'class':'entry-title-inner'})
                feature = featureHTML.string
                if(feature == None):
                    featureHTML.find(True).replaceWith('')
                    featureHTML = featureOriginal.find("span",{'class':'entry-title-inner'}).contents[0]
                    feature = featureHTML.string
                feature = re.sub("[.]", "", feature)
                feature = re.sub(" \(", "_(", feature)
                feature = feature.replace("\\","")
                featureSplit = feature.split("_")
                if(featureSplit[0]=="Legendary Resistance"):
                    legendaryRes = featureSplit[1][1]
                else:
                    features.append(featureSplit[0])
                if(featureSplit[0]=="Spellcasting"):
                    spellCaster = True
            currentIndexOfInterest += 1

        if(action):
            actionList = placesOfInterest[currentIndexOfInterest]
            actionList = actionList.find_all("div",{'class': ['rd__b',"rd__b--3"]})
            actionNames = []
            for i, actionOriginal in enumerate(actionList):
                actionEntry = {}
                #Get action name
                actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'})
                actionName = actionNameHTML.string
                if(actionName == None):
                    actionNameHTML.find(True).replaceWith('')
                    actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'}).contents[0]
                    actionName = actionNameHTML.string
                actionName = re.sub("[.]", "", actionName)
                actionName = re.sub(" \(", "_(", actionName)
                actionName = actionName.replace("\\","")
                actionSplit = actionName.split("_")
                actionName = actionSplit[0]
                if(actionName == "Spellcasting"):
                    spellCaster = True
                elif(actionName == "Multiattack"):
                    multiAttack = True
                    continue
                actionNames.append(actionName)

                actionEntry["name"] = actionName
                
                toHit = actionOriginal.find("span",{"class":"rd__roller--roll-prof-bonus"})
                if(toHit != None):
                    actionEntry['toHit'] = toHit.string
                
                
                importantStats = actionOriginal.find_all(has_dice)
                damageIncrement = 0
                for stat in importantStats:
                    if '"subtype":"damage"' in str(stat) or 'subType&quot;:&quot;damage' in str(stat):
                        damageFormula = stat.string.replace(" ","")
                        diceRolls = re.search(r"[\d]+d[\d]+",damageFormula).group()
                        try:
                            modifier = re.search(r"[+-]\d+",damageFormula).group()
                        except:
                            modifier = 0
                        
                        PB = 0
                        if('PB' in damageFormula):
                            #FIX THIS BEFORE CALLING IT GOOD
                            pass
                        
                        numDice,diceType = diceRolls.split("d")
                        averagePerDie = (1 + int(diceType))/2
                        average = int(averagePerDie * int(numDice))
                        damageTotal = average + int(modifier) + PB

                        damageIncrement += 1
                        actionEntry['damage'+str(damageIncrement)]=damageTotal
                
                typeIncrement = 0
                dtypes = re.findall(r'\) [\S]+ damage',str(actionOriginal))
                for dtype in dtypes:
                    if(dtype == "half" or dtype == "full"):
                        continue
                    typeIncrement += 1
                    actionEntry['dtype' + str(typeIncrement)] = dtype.split(" ")[1]
                
                dc = actionOriginal.find("span",{"class":"rd__dc--rollable-text"})
                if(dc != None and actionName != "Spellcasting"):
                    actionEntry['DC'] = dc.string
                    
                appliedConditions = re.findall(conditionsRegex,str(actionOriginal))
                abilityConditions = []
                for appliedCondition in appliedConditions:
                    if(appliedCondition not in abilityConditions):
                        abilityConditions.append(appliedCondition)
                actionEntry["conditions"] = abilityConditions
                if("Spellcasting" in actionNames):
                    spellCaster = True
                else:
                    actions.append(actionEntry)
                
            if(multiAttack):
                multiAttackText = actionList[0]
                multiAttackText = multiAttackText.find("p").string
                match = re.search(r"level",str(multiAttackText))
                
                if(match==None):
                    regex = "one|two|three|four|five|six|seven|each"
                    if("Spellcasting" in actionNames):
                        actionNames.pop(actionNames.index("Spellcasting"))

                    for actionName in actionNames:
                        if " " in actionName:
                            newActionNameSplit = actionName.split(" ")
                            regexPiece = ""
                            for part in newActionNameSplit:
                                regexPiece += "[" + part[0].lower() + part[0].upper() + "]" + part[1:]
                                if(newActionNameSplit.index(part)<len(newActionNameSplit)-1):
                                    regexPiece += " "
                        else:
                            regexPiece = "[" + actionName[0].lower() + actionName[0].upper() + "]" + actionName[1:]
                        regex += "|"+ regexPiece
                    matches = re.findall(re.compile(regex),str(multiAttackText))
                    matches = [x.lower() for x in matches]
#                     print(name,matches,regex,actionNames)
                    multiattackArr = {'name':"Multiattack"}
    #                 print(name, matches)
                    if("each" in matches):
                        indexMatch = matches.index("each")
                        numTimes = 1
                        try:
                            if(matches[indexMatch-1] in list(num_dict.keys())):
                                numTimes = int(num_dict[matches[indexMatch-1]])
                        except:
                            pass
                        for actionName in actionNames:
                            multiattackArr[actionName] = numTimes

                    else:
                        for actionName in actionNames:
                            try:
                                indexMatch = matches.index(actionName.lower())
                                
                                if(matches[indexMatch-1] in list(nums_dict.keys())):
                                    multiattackArr[actionName] = int(nums_dict[matches[indexMatch-1]])
                                else:
                                    multiattackArr[actionName] = 1
                            except:
                                pass
    #                 print(name,multiattackArr)
                    if len(multiattackArr.keys())==1:
                        multiattackArr["complicated"] = True
                    else:
                        multiattackArr["complicated"] = False
                    actions.append(multiattackArr)
                    
                    
                
            currentIndexOfInterest += 1

            
        if(bonus):
            actionList = placesOfInterest[currentIndexOfInterest]
            actionList = actionList.find_all("div",{'class': ['rd__b',"rd__b--3"]})
            actionNames = []
            for i, actionOriginal in enumerate(actionList):
                actionEntry = {}
                #Get action name
                actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'})
                actionName = actionNameHTML.string
                if(actionName == None):
                    actionNameHTML.find(True).replaceWith('')
                    actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'}).contents[0]
                    actionName = actionNameHTML.string
                actionName = re.sub("[.]", "", actionName)
                actionName = re.sub(" \(", "_(", actionName)
                actionName = actionName.replace("\\","")
                actionSplit = actionName.split("_")
                actionName = actionSplit[0]
                if(actionName == "Spellcasting"):
                    spellCaster = True
                elif(actionName == "Multiattack"):
                    multiAttack = True
                    continue
                actionNames.append(actionName)

                actionEntry["name"] = actionName
                
                toHit = actionOriginal.find("span",{"class":"rd__roller--roll-prof-bonus"})
                if(toHit != None):
                    actionEntry['toHit'] = toHit.string
                
                
                importantStats = actionOriginal.find_all(has_dice)
                damageIncrement = 0
                for stat in importantStats:
                    if '"subtype":"damage"' in str(stat) or 'subType&quot;:&quot;damage' in str(stat):
                        damageFormula = stat.string.replace(" ","")
                        diceRolls = re.search(r"[\d]+d[\d]+",damageFormula).group()
                        try:
                            modifier = re.search(r"[+-]\d+",damageFormula).group()
                        except:
                            modifier = 0
                        
                        PB = 0
                        if('PB' in damageFormula):
                            #FIX THIS BEFORE CALLING IT GOOD
                            pass
                        
                        numDice,diceType = diceRolls.split("d")
                        averagePerDie = (1 + int(diceType))/2
                        average = int(averagePerDie * int(numDice))
                        damageTotal = average + int(modifier) + PB

                        damageIncrement += 1
                        actionEntry['damage'+str(damageIncrement)]=damageTotal
                
                typeIncrement = 0
                dtypes = re.findall(r'\) [\S]+ damage',str(actionOriginal))
                for dtype in dtypes:
                    if(dtype == "half" or dtype == "full"):
                        continue
                    typeIncrement += 1
                    actionEntry['dtype' + str(typeIncrement)] = dtype.split(" ")[1]
                
                dc = actionOriginal.find("span",{"class":"rd__dc--rollable-text"})
                if(dc != None and actionName != "Spellcasting"):
                    actionEntry['DC'] = dc.string
                    
                appliedConditions = re.findall(conditionsRegex,str(actionOriginal))
                abilityConditions = []
                for appliedCondition in appliedConditions:
                    if(appliedCondition not in abilityConditions):
                        abilityConditions.append(appliedCondition)
                actionEntry["conditions"] = abilityConditions
                if("Spellcasting" in actionNames):
                    spellCaster = True
                else:
                    bonuses.append(actionEntry)
            currentIndexOfInterest += 1
        if(reaction):
            actionList = placesOfInterest[currentIndexOfInterest]
            actionList = actionList.find_all("div",{'class': ['rd__b',"rd__b--3"]})
            actionNames = []
            for i, actionOriginal in enumerate(actionList):
                actionEntry = {}
                #Get action name
                actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'})
                actionName = actionNameHTML.string
                if(actionName == None):
                    actionNameHTML.find(True).replaceWith('')
                    actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'}).contents[0]
                    actionName = actionNameHTML.string
                actionName = re.sub("[.]", "", actionName)
                actionName = re.sub(" \(", "_(", actionName)
                actionName = actionName.replace("\\","")
                actionSplit = actionName.split("_")
                actionName = actionSplit[0]
                if(actionName == "Spellcasting"):
                    spellCaster = True
                elif(actionName == "Multiattack"):
                    multiAttack = True
                    continue
                actionNames.append(actionName)

                actionEntry["name"] = actionName
                
                toHit = actionOriginal.find("span",{"class":"rd__roller--roll-prof-bonus"})
                if(toHit != None):
                    actionEntry['toHit'] = toHit.string
                
                
                importantStats = actionOriginal.find_all(has_dice)
                damageIncrement = 0
                for stat in importantStats:
                    if '"subtype":"damage"' in str(stat) or 'subType&quot;:&quot;damage' in str(stat):
                        damageFormula = stat.string.replace(" ","")
                        diceRolls = re.search(r"[\d]+d[\d]+",damageFormula).group()
                        try:
                            modifier = re.search(r"[+-]\d+",damageFormula).group()
                        except:
                            modifier = 0
                        
                        PB = 0
                        if('PB' in damageFormula):
                            #FIX THIS BEFORE CALLING IT GOOD
                            pass
                        
                        numDice,diceType = diceRolls.split("d")
                        averagePerDie = (1 + int(diceType))/2
                        average = int(averagePerDie * int(numDice))
                        damageTotal = average + int(modifier) + PB

                        damageIncrement += 1
                        actionEntry['damage'+str(damageIncrement)]=damageTotal
                
                typeIncrement = 0
                dtypes = re.findall(r'\) [\S]+ damage',str(actionOriginal))
                for dtype in dtypes:
                    if(dtype == "half" or dtype == "full"):
                        continue
                    typeIncrement += 1
                    actionEntry['dtype' + str(typeIncrement)] = dtype.split(" ")[1]
                
                dc = actionOriginal.find("span",{"class":"rd__dc--rollable-text"})
                if(dc != None and actionName != "Spellcasting"):
                    actionEntry['DC'] = dc.string
                    
                appliedConditions = re.findall(conditionsRegex,str(actionOriginal))
                abilityConditions = []
                for appliedCondition in appliedConditions:
                    if(appliedCondition not in abilityConditions):
                        abilityConditions.append(appliedCondition)
                actionEntry["conditions"] = abilityConditions
                if("Spellcasting" in actionNames):
                    spellCaster = True
                else:
                    reactions.append(actionEntry)
            currentIndexOfInterest += 1
            
        #We don't care about gathering the info in these, though we leave these here just in case
        #we care about trying to access the associated elements of information at a later date
        if(legendaryAct!=0):
            currentIndexOfInterest += 1
        if(mythic):
            currentIndexOfInterest += 1
        if(lair):
            currentIndexOfInterest += 1
        if(regional):
            currentIndexOfInterest += 1
        if(spellCaster):
            casters += 1


In [122]:
print(actions,bonuses,reactions)

[{'name': 'Pseudopod', 'toHit': '+13', 'damage1': 15, 'damage2': 9, 'dtype1': 'force', 'dtype2': 'poison', 'conditions': []}, {'name': 'Multiattack', 'Pseudopod': 3, 'complicated': False}] [{'name': 'Infestation Spores', 'DC': '19', 'conditions': []}, {'name': 'Mind Control Spores', 'DC': '19', 'conditions': ['charmed']}] [{'name': 'Protective Thrall', 'conditions': ['charmed']}]
